In [ ]:
!pip install -q streamlit
!pip install -q streamlit_option_menu
!npm install localtunnel

In [ ]:
%%writefile app.py
import streamlit as st
from streamlit_option_menu import option_menu
import pandas as pd
import numpy as np
import joblib


# pipe_knn = joblib.load('knn.pkl')
# pipe_bagging = joblib.load('bagging.pkl')
# pipe_dence = joblib.load('dence.pkl')


# def neuro_result(X: np.ndarray) -> np.ndarray:
#     return np.array([np.argmax(pred) for pred in X], dtype=bool)


# models = {
#     "KNN": pipe_knn,
#     "Bagging": pipe_bagging,
#     "Нейронная сеть": pipe_dence
# }


# def upload(model_option):
#     uploaded_file = st.file_uploader("Выберите файл .csv (без заголовков столбцов):")

#     if uploaded_file:
#         upload_data = pd.read_csv(uploaded_file, sep=";", encoding='utf-8')
#         upload_data.columns = ["est_diameter_min", "est_diameter_max", "relative_velocity", "miss_distance", "absolute_magnitude"]

#     if st.button("Получить предсказание", key='1'):
#         if not uploaded_file:
#             st.warning('Сначала нужно загрузить данные!', icon="⚠️")
#         else:
#             y_pred = models[model_option].predict(upload_data)
#             if model_option == "Нейронная сеть":
#                 y_pred = neuro_result(y_pred)
#             upload_data['hazardous'] = y_pred
#             st.write(upload_data)
#             st.download_button("Сохранить результат", upload_data.to_csv(index=False, encoding='utf-8'), "file.csv", "text/csv", key='download-csv')


# def input(model_option):
#     edited_df = st.experimental_data_editor(pd.DataFrame([{
#         "est_diameter_min": "",
#         "est_diameter_max": "",
#         "relative_velocity": "",
#         "miss_distance": "",
#         "absolute_magnitude": ""
#     }]), num_rows="dynamic", use_container_width=False, width = 1000)

#     if st.button("Получить предсказание", key='2'):
#         try:
#             y_pred = models[model_option].predict(edited_df)
#             if model_option == "Нейронная сеть":
#                 y_pred = neuro_result(y_pred)
#             edited_df['hazardous'] = y_pred
#             st.write(edited_df)
#             st.download_button("Сохранить результат", edited_df.to_csv(index=False, encoding='utf-8'), "file.csv", "text/csv", key='download-csv')
#         except:
#             st.warning('Некорректные данные!', icon="⚠️")


def prediction():
    model_option = st.selectbox('Выберите модель для предсказания', ('MODEL1', 'MODEL2', 'MODEL3'))
    tab1, tab2 = st.tabs(["Загрузка файла .csv", "Ручной ввод значений"])

    with tab1:
        upload(model_option)

    with tab2:
        input(model_option)


INFO = r"""
## Описание входных данных (объекты)

Данные, конечно, - это хорошо, но правильные данные - еще лучше! Чтобы получить достоверное (или не очень) предсказание, разберемся, в чем заключается смысл данных, и какие объекты нужно подавать на вход модели машинного обучения:

|  Название столбца  | Описание                                   |
| ------------------ | ------------------------------------------ |
| 1   | 2 |
| 3   | 4 |


## Описание выходных данных (предсказания)

Предсказываться будет значение $<y> \in $ {$0, 1$} - опасность объекта
(0, если объект не представляет опасности, и 1, если представляет).

Таким образом, решается задача бинарной классификации.

## Выбранные модели машинного обучения

А теперь пора представить наших героев, или антигероев... модели машинного обучения!

В данном приложении используются 3 вида моделей: 
- линейные (KNN);
- ансамбли (BaggingClassifier);
- нейронные сети (многослойная полносвязная НС прямого распространения).

### KNN

KNN, или же метод k-ближайших соседей, являетмя одним из самых простых,
но в то же время эффективных классификаторов. Он исходит из так называемой "гипотезы компактности",
которая гласит о том, что объекты одного класса находятся близко к друг другу,
а объекты разных классов, навпротив, - далеко друг от друга. 

Когда мы говорим "близко" или "далеко", то имеем ввиду значение функции-расстояния $\rho(x^1, x^2)$,
то есть такой функции, которая показывает меру близости объектов $x^1$ и $x^2$ из нашего признакового пространства $X$.  

Функция-расстояния должна удовлетворять трем условиям:

1. $\rho(x, x) = 0$;
2. $\rho(x^1, x^2) = \rho(x^2, x^1)$;
3. $\rho(x^1, x^3) < \rho(x^1, x^2) + \rho(x^2, x^3)$.

Но все же вернемся к KNN. Задачется параметр $k$ - количество ближайших соседей. Когда нам поступает объект для классификации, мы смотрим расстояние от этого объекта до каждого из обучающей выборки. Выбираем из этих расстояний $k$ наименьших -- самые близкие объекты к нашему. И определяем, каких меток класса больше среди тех самых отобранных "соседей". Максимальная метка и будет нашим ответом.

![knn](https://www.newtechdojo.com/wp-content/uploads/2020/06/KNN-1.gif)

### BaggingClassifier

Данный вид ансамблевых алгоритмов использует $n$ одинаковых моделей,
предварительно настроенных, которые задаются в качестве гиперпараметра.
При помощи техники бутстрапирования из тренировочной выборки размерности $l \times d$ случайно выбирается $l$ объектов с повторением
(то есть объекты могут повторяться в подвыборке), и такая подвыборка генерируется для кажой модели внутри BaggingClassifier.

В результате, когда нам приходят данные для предсказания, мы передаем их в каждую обученную на "разных" данных модель и их результат усредняем
(либо берем среднее, если задача регрессии, либо берем моду, если задача классификации). ГОЛОСОВАНИЕ!

![](https://media.geeksforgeeks.org/wp-content/uploads/20210707140912/Bagging.png)

### Нейронная сеть (многослойная полносвязная НС прямого распространения)

Глобально, такой вид нейронных сетей представляет из себя последовательное матричное умножение
исходной матрицы признаков на матрицу коэффициентов слоя
(матрица связи каждого нейрона текущего слоя с каждым нейроном следующего),
добавляя матрицу смещений и функцию активации к получившемуся результату.
И так делается до тех пор, пока мы не пройдем все слои (выход предыдущего слоя - это вход текущего).

Можно рассматривать НС как вычислительный граф, и тогда все становится на свои места.

![НС](https://neerc.ifmo.ru/wiki/images/9/97/Single-layer-neural-net-scheme.png)
        
"""


def description():
    st.markdown(INFO)


pages = {
    'Описание': description,
    'Предсказание': prediction,
    'Визуализации': None,
}


def main():
    with st.sidebar:
        selected = option_menu(
            "Меню",
            ["Описание", 'Предсказание', 'Визуализации'],
            icons=['info-circle', 'bar-chart', 'tag'],
            menu_icon="cast",
            default_index=0
        )

    pages[selected]()


if __name__ == '__main__':
    main()


In [ ]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com

In [ ]:
!npx localtunnel --port 8501
# перейти по ссылке и вставить айпишник из верхней ячейки

In [ ]:
# # для очистки портов, Гуня сказал, что можно использовать для перезагрузки дашборда
# ! fuser -k 8501/tcp